In [ ]:
from pathlib import Path
from datetime import datetime

import numpy as np
import pandas as pd
import xarray as xr
import rioxarray as rxa
from rasterio.enums import Resampling
import matplotlib.pyplot as plt

import sys
sys.path.append('/Users/zmhoppinen/Documents/nisar_swe/projects/reference_points')
from swe_retrieval import guneriussen_phase_from_depth, guneriussen_depth_from_phase, density_to_permittivity, rho_250
from resolution import coarsen_to_resolution
from constants import NISAR


In [ ]:
nisar = NISAR()

target_resolution = nisar.multi_look_resolution # match nisar

In [3]:
sd_fps = sorted(list(Path('/Users/zmhoppinen/Documents/nisar_swe/data/lidar/nani').glob('*_SNOWDEPTH.tif')))
print(len(sd_fps))
sds = []
for i, fp in enumerate(sd_fps):
    # drop Feb 13 2024 due to big gaps
    if fp.stem.split('_')[1] == '20240213': continue
    da = xr.open_dataarray(fp).expand_dims(time = [datetime.strptime(fp.stem.split('_')[1], '%Y%m%d')]).squeeze('band', drop = True)
    sds.append(da)

for year in range(2021, 2025):
    sds.append(xr.zeros_like(sds[0]).isel(time = 0).expand_dims(time = [datetime.strptime(f'{year}1001', '%Y%m%d')]))

17


In [4]:
mores = xr.concat(sds, 'time').sortby('time')#[:, ::20, ::20]
# mores = mores.where(mores.isnull().sum('time') < 16)
mores = mores.where(mores >= 0)

inc = xr.open_dataarray('/Users/zmhoppinen/Documents/nisar_swe/data/uavsar/generated/052/inc_mores.tif')[0].rio.reproject_match(mores[0])

mores_coarse = coarsen_to_resolution(mores, res = target_resolution)
inc_coarse = np.deg2rad(coarsen_to_resolution(inc, res = target_resolution))

In [5]:
unw = guneriussen_phase_from_depth(mores_coarse.diff('time'), epsilon = rho_250, inc = inc_coarse)

wrapped = xr.zeros_like(unw)
wrapped.data = np.angle(np.exp(1j * unw))

In [6]:
phase = guneriussen_phase_from_depth(mores_coarse.diff('time'), epsilon=rho_250, inc=inc_coarse)
recovered = guneriussen_depth_from_phase(phase, rho_250, inc=inc_coarse)

print("Mean diff:", np.nanmean(mores_coarse.diff('time').values - recovered.values))
print("Max abs diff:", np.nanmax(np.abs(mores_coarse.diff('time').values - recovered.values)))


Mean diff: -1.0046377165048572e-09
Max abs diff: 1.3218988357266426e-06


In [7]:
tau = xr.open_dataarray('/Users/zmhoppinen/Documents/nisar_swe/data/uavsar/generated/052/tau_2x8.tif')[0].rio.reproject_match(mores[0])
ginf = xr.open_dataarray('/Users/zmhoppinen/Documents/nisar_swe/data/uavsar/generated/052/ginf_2x8.tif')[0].rio.reproject_match(mores[0])

r2 = xr.open_dataarray('/Users/zmhoppinen/Documents/nisar_swe/data/uavsar/generated/052/r2_2x8.tif')[0].rio.reproject_match(mores[0])

tau_coarse = coarsen_to_resolution(tau, res = target_resolution)
ginf_coarse = coarsen_to_resolution(ginf, res = target_resolution)
r2_coarse = coarsen_to_resolution(r2, res = target_resolution)

In [8]:
ds = mores_coarse.to_dataset(name = 'sds')

In [9]:
ds['int'] = wrapped
ds['unw'] = unw
ds['inc'] = inc_coarse
ds['tau'] = tau_coarse
ds['ginf'] = ginf_coarse
ds['r2'] = r2_coarse

In [10]:
ds = ds.assign_coords(ref_time = ("time", np.concatenate([[np.datetime64('NaT')], ds.time.data[:-1]])))
ds = ds.assign_coords(sec_time = ("time", np.concatenate([[np.datetime64('NaT')], ds.time.data[1:]])))

# make first (would be nan since no difference) 12 day instead to match nisar
dt = np.concatenate([[12], [pd.Timedelta(i).days for i in ds.time.data[1:] - ds.time.data[:-1]]])
ds = ds.assign_coords(temp_baseline = ("time", dt))

In [11]:
import xarray as xr
import numpy as np

def coherence_model(dt, g_inf, tau):
    return (1 - g_inf) * np.exp(-dt / tau) + g_inf

# Apply across the dataset
gamma_model = xr.apply_ufunc(
    coherence_model,
    ds["temp_baseline"],       # (time,)
    ds["ginf"],    # (y, x)
    ds["tau"],      # (y, x)
    input_core_dims=[["time"], [], []],  # dt varies along time
    output_core_dims=[["time"]],
    vectorize=True,
    dask="parallelized",  # allows lazy/dask if dataset is big
    output_dtypes=[float],
)

# Save result back into dataset
ds["coherence"] = gamma_model.transpose('time', 'y', 'x')


In [12]:
ds['diff'] = ds['sds'].diff('time')

In [ ]:
import numpy as np
import xarray as xr

def fit_predict_12day(dsd: xr.DataArray) -> xr.DataArray:
    """
    For each pixel (y, x), fit a linear regression of dsd vs temporal_baseline,
    excluding baselines > 90 days and times when the total snow decreased then predict snow depth change at 12 days.
    Finally mask pixels that have more than 3 nans in the total stack.
    
    Parameters
    ----------
    dsd : xr.DataArray
        Dimensions: (time, y, x)
        Coordinates: temporal_baseline (on 'time')
    
    Returns
    -------
    xr.DataArray
        Dimensions: (y, x)
        Predicted dsd at 12 days.
    """
    # mask out > 60 day baselines
    mask = dsd['temp_baseline'] <= 90
    dsd = dsd.where(mask, drop=True)

    mask = dsd.mean(['x','y']) >= 0
    dsd = dsd.where(mask, drop=True)
    
    # extract the baseline values
    tb = dsd['temp_baseline']
    print(len(tb))
    
    # function to apply per pixel
    def linregress_pixel(yvals, tbvals):
        # drop NaNs
        m = np.isfinite(yvals) & np.isfinite(tbvals)
        if m.sum() < 2:  # not enough points to fit
            return np.nan
        coeffs = np.polyfit(tbvals[m], yvals[m], 1)  # slope, intercept
        return np.polyval(coeffs, 12.0)  # prediction at 12 days

    # use apply_ufunc for pixelwise regression
    result = xr.apply_ufunc(
        linregress_pixel,
        dsd, tb,
        input_core_dims=[["time"], ["time"]],
        vectorize=True,
        dask="parallelized",
        output_dtypes=[float],
    )
    
    result.name = "dsd_12day"
    result.attrs["description"] = "Predicted snow depth change at 12 days from linear fit"

    result = result.where(dsd.isnull().sum('time')<3)
    
    return result

ds['dsd_12'] = fit_predict_12day(ds['diff'])


9


In [14]:
ds.to_netcdf(f'/Users/zmhoppinen/Documents/nisar_swe/data/reference_point/full_{target_resolution}.nc')